In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import folium
from haversine import haversine, Unit
#import pyodbc

In [ ]:
# Set the directory path
dir_path = 'Data/street'

def read_files(path):
    """Read in all CSV files in a directory and concatenate them into a single dataframe"""
    # List all CSV files in the directory
    file_names = [f for f in os.listdir(path) if f.endswith('.csv')]
    
    # Read each CSV file into a separate dataframe
    dfs = []
    for file_name in file_names:
        print(f'Reading file: {file_name}')
        file_path = os.path.join(path, file_name)
        df1 = pd.read_csv(file_path)
        dfs.append(df1)
    
    # Concatenate all dataframes into a single dataframe
    df_combined = pd.concat(dfs, ignore_index=True)
    
    return df_combined

# Call the read_files function to read in all CSV files in the specified directory
df = read_files(dir_path)

In [ ]:
df.head(10)

In [ ]:
len(df)

In [ ]:
df.info()

In [ ]:
df[df['Crime ID'].isnull()]

In [ ]:
sum(df['Crime ID'].isnull())

In [ ]:
df[df['Longitude'].isnull()]

In [ ]:
sum(df['Longitude'].isnull()), sum(df['Latitude'].isnull())

In [ ]:
police_centers = df['Falls within'].unique()
police_centers

In [ ]:
len(police_centers)

In [ ]:
sum(df['Falls within'].isnull())

In [ ]:
df['Falls within'].value_counts()

In [ ]:
# Calculate the distribution of values in 'my_column'
value_counts = df['Falls within'].value_counts().head(15)

# Create a larger figure
plt.figure(figsize=(15,10))

# Plot the distribution as a bar chart
plt.bar(value_counts.index, value_counts.values)
plt.xlabel('Value', fontsize=12)
plt.ylabel('Frequency', fontsize=12)

# Add frequency counts as text labels on top of each bar
for i, v in enumerate(value_counts.values):
    plt.text(i, v, str(v), color='black', fontweight='bold', ha='center', fontsize=10)

# Tilt the x-axis labels
plt.xticks(rotation=45, ha='right')

# Add a title to the plot
plt.title('The distribution of crimes within the police centers', fontsize=14)

# Show the plot
plt.show()

In [ ]:
crime_types = df['Crime type'].unique()
crime_types

In [ ]:
len(crime_types)

In [ ]:
sum(df['Crime type'].isnull())

In [ ]:
df.isnull().sum()

In [ ]:
cleaned_data = df.dropna(subset=['Longitude'])
cleaned_data

In [ ]:
len(cleaned_data)

In [ ]:
del df

In [ ]:
# Convert 'Latitude' and 'Longitude' columns to a NumPy array
coords = cleaned_data[['Latitude', 'Longitude']].to_numpy()

# Define the latitude and longitude of London
london = (51.509865, -0.118092)

index = len(coords)
distance = np.zeros(index)

for row, point in enumerate(coords):
    distance[row] = haversine(london, point, unit=Unit.KILOMETERS)

    if row % 1_000_000 == 0:
        print(row,'rows got calculated')

In [ ]:
mask = distance <= 100

london_df = cleaned_data[mask].copy()
london_df['Distance_to_London'] = distance[mask]

london_df

In [ ]:
del cleaned_data

In [ ]:
# Plot histogram of the 'my_column' column
london_df['Distance_to_London'].hist(bins=25)

# Add labels and title to the plot
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Histogram of my_column')

# Show the plot
plt.show()

In [ ]:
len(london_df)

In [ ]:
# store the dataframe into a database
from sqlalchemy import create_engine

# Set up database connection
engine = create_engine('sqlite:///london_crime.db', echo=False)

# Define chunk size
chunk_size = 1_000_000

# Insert data into database in chunks
j = 1
for i in range(0, len(london_df), chunk_size):
    chunk = london_df.iloc[i:i+chunk_size]
    chunk.to_sql('street_data', con=engine, if_exists='append', index=False)
    print(j*chunk_size, ' rows got inserted')
    j += 1

# Close the database connection and dispose of the engine
engine.dispose()

In [ ]:
london_df[london_df['Distance_to_London'] <= 10]

In [ ]:
map_new=folium.Map()
list_coor=london_df[['Longitude','Latitude']].head(100).values.tolist()
for i in list_coor:
    map_new.add_child(folium.Marker(location=[i[1],i[0]],icon=folium.Icon(color='green')))

In [ ]:
map_new.save('Uk_crime.html')

In [ ]:
map_new